In [1]:
import numpy as np
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from standardizer import normate_dataset_period
import datetime as datetime
from tsfresh.feature_extraction import extract_features
from tsfresh.feature_extraction import MinimalFCParameters
from preprocessor import preprocessor
import os

# Data Loading and Pre-Processing

In [12]:
#Get data file names
data_paths = [ './data/'+file for file in os.listdir("./data/") if file.split('_')[0] + file.split('_')[1]=='punchnorm' and file.split('_')[-1]=='20000.csv' ]
raw_data_path = "./data/complete_smartpunch_dataset_7606punches.json"
data_paths.insert(0, raw_data_path)

print(data_paths)



['./data/punch_norm_1000_20000.csv', './data/punch_norm_1300_20000.csv', './data/punch_norm_1500_20000.csv', './data/punch_norm_1800_20000.csv', './data/punch_norm_2000_20000.csv', './data/punch_norm_2200_20000.csv', './data/punch_norm_2500_20000.csv']


# Training Loop

In [9]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score 
from sklearn.preprocessing import MinMaxScaler


#store intermediary result dicts in list
results_complete = []


for row_idx,data_path in enumerate(data_paths):
    
    
    #if data is the raw data file
    if data_path.split('.')[-1]=='json':
        processor = preprocessor(filepath=data_path, data_stage='raw')
        processor.pre_process()
        period_length = 0
        sampling_rate = 0
    
    #else data is already processed
    else:
        processor = preprocessor(filepath=data_path, data_stage='processed')
        period_length = int( data_path.split('_')[2] )
        sampling_rate = int( data_path.split('_')[3].split('.')[0] )
        
    features = processor.extract_features()
    labels = processor.get_labels()
    
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)
    norm = MinMaxScaler().fit(X_train)
    
    X_train = norm.transform(X_train)
    X_test = norm.transform(X_test)
    
    intermediary_results = []
    
    

    #Random Forest
    max_depth = [int(x) for x in np.linspace(start=20, stop=100, num=5)]
    n_estimators = [20,50,80,120,200,500,800]
    param_grid_forest = {'max_depth':max_depth,
                'n_estimators':n_estimators,
                'criterion':['gini', 'entropy', 'log_loss']}

    search_forest = GridSearchCV(RandomForestClassifier(), param_grid_forest, cv=5, verbose=1)
    search_forest.fit(X_train, y_train)
    
    max_depth = search_forest.best_params_['max_depth']
    n_estimators = search_forest.best_params_['n_estimators']
    criterion = search_forest.best_params_['criterion']
    
    best_forest = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, criterion=criterion)
    best_forest.fit(X_train, y_train)
    y_pred = best_forest.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    
    
    
    results = {'classifier':'RandomForestClassifier',
               'period_length':period_length,
               'sampling_rate':sampling_rate,
               'accuracy':accuracy,
               'precision':precision,
               'recall':recall,
               'f1_score':f1,
               'best_params_': str(search_forest.best_params_)
              }
    
    intermediary_results.append(results)
    
    print(results)
    
    
    
    
    
    #Logistic Regression
    C_values = [0.001, 0.01,0.1, 1, 10, 100]
    param_grid_logreg = {'C':C_values}

    search_logreg = GridSearchCV(LogisticRegression(), param_grid_logreg, cv=5, verbose=1)
    search_logreg.fit(X_train, y_train)
    
    C_value = search_logreg.best_params_['C']
    
    best_logreg = LogisticRegression(C=C_value)
    best_logreg.fit(X_train, y_train)
    y_pred = best_logreg.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    
    
    
    results = {'classifier':'LogisticRegression',
               'period_length':period_length,
               'sampling_rate':sampling_rate,
               'accuracy':accuracy,
               'precision':precision,
               'recall':recall,
               'f1_score':f1,
               'best_params_': str(search_logreg.best_params_)
              }
    
    intermediary_results.append(results)
    
    print(results)
    
    
    
    #Naive Bayes
    param_grid_nb = {'var_smoothing': np.logspace(0,-9, num=10)}
    
    search_nb = GridSearchCV(GaussianNB(), param_grid_nb, cv=5, verbose=1)
    search_nb.fit(X_train, y_train)
    
    var_smoothing = search_nb.best_params_['var_smoothing']
    
    best_nb = GaussianNB(var_smoothing=var_smoothing)
    best_nb.fit(X_train, y_train)
    y_pred = best_nb.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    
    
    
    results = {'classifier':'GaussianNB',
               'period_length':period_length,
               'sampling_rate':sampling_rate,
               'accuracy':accuracy,
               'precision':precision,
               'recall':recall,
               'f1_score':f1,
               'best_params_': str(search_nb.best_params_)
              }
    
    intermediary_results.append(results)
    
    print(results)
    

    
    #AdaBoost
    tree = DecisionTreeClassifier
    estimators = [tree(max_depth=1), tree(max_depth=2), tree(max_depth=3)]
    n_estimators = [int(x) for x in np.linspace(start=20, stop=200, num=5)]
    learning_rates = [0.1, 1, 10]
    
    param_grid_adaboost = {'estimator':estimators, 'n_estimators':n_estimators,
                           'learning_rate':learning_rates}

    
    search_adaboost = GridSearchCV(AdaBoostClassifier(), param_grid_adaboost, cv=5, verbose=1)
    search_adaboost.fit(X_train, y_train)
    
    estimator = search_adaboost.best_params_['estimator']
    n_estimators = search_adaboost.best_params_['n_estimators']
    learning_rate = search_adaboost.best_params_['learning_rate']
    
    
    best_adaboost = AdaBoostClassifier(estimator=estimator, n_estimators = n_estimators, learning_rate = learning_rate)
    best_adaboost.fit(X_train, y_train)
    y_pred = best_adaboost.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    results = {'classifier':'AdaBoostClassifier',
               'period_length':period_length,
               'sampling_rate':sampling_rate,
               'accuracy':accuracy,
               'precision':precision,
               'recall':recall,
               'f1_score':f1,
               'best_params_': str(search_adaboost.best_params_)
              }
    
    intermediary_results.append(results)
    
    
    #SVC
    C = [0.001, 0.01, 0.1, 1, 10, 100]
    kernels = ['sigmoid', 'linear', 'rbf']
    param_grid_svm = {'C':C,
                     'kernel':kernels}

    search_svm = GridSearchCV(SVC(), param_grid_svm, cv=5, verbose=1)
    search_svm.fit(X_train, y_train)
    
    C = search_svm.best_params_['C']
    kernel = search_svm.best_params_['kernel']
    
    best_svm = SVC(C=C, kernel=kernel)
    best_svm.fit(X_train, y_train)
    y_pred = best_svm.predict(X_test)
    
       
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    
    
    
    results = {'classifier':'SVC',
               'period_length':period_length,
               'sampling_rate':sampling_rate,
               'accuracy':accuracy,
               'precision':precision,
               'recall':recall,
               'f1_score':f1,
               'best_params_': str(search_svm.best_params_)
              }
    
    intermediary_results.append(results)
    
    print(results)
    
    
    
    
    #End of Loop
    results_complete.extend(intermediary_results)
    
    intermediary_results_df = pd.DataFrame(intermediary_results, index=[0,1,2,3,4])
    intermediary_results_df.to_csv('./new_results/ml_results_{}_{}.csv'.format(period_length, sampling_rate))
    
    
    
    
    
#Save results
print(results_complete)

#results_complete_df = pd.DataFrame(results_complete, index=np.arange(0, (row_idx+1)*5))
#results_complete_df.to_csv('./new_results/ml_results_complete.csv')

    
    
    
    
    
    

Feature Extraction: 100%|██████████| 20/20 [00:15<00:00,  1.29it/s]


Fitting 5 folds for each of 105 candidates, totalling 525 fits
{'classifier': 'RandomForestClassifier', 'period_length': 2200, 'sampling_rate': 5000, 'accuracy': 0.9842312746386334, 'precision': 0.9842622409661516, 'recall': 0.9844661164885883, 'f1_score': 0.9843416633691274, 'best_params_': "{'criterion': 'log_loss', 'max_depth': 40, 'n_estimators': 120}"}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\dante\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dante\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

C:\Users\dante\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'classifier': 'LogisticRegression', 'period_length': 2200, 'sampling_rate': 5000, 'accuracy': 0.9145860709592641, 'precision': 0.9156464883250598, 'recall': 0.9159664944175377, 'f1_score': 0.9157689437905774, 'best_params_': "{'C': 100}"}
Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'classifier': 'GaussianNB', 'period_length': 2200, 'sampling_rate': 5000, 'accuracy': 0.7385019710906702, 'precision': 0.779362117981521, 'recall': 0.7474488348284496, 'f1_score': 0.6842903720155114, 'best_params_': "{'var_smoothing': 0.001}"}
Fitting 5 folds for each of 45 candidates, totalling 225 fits


...

Feature Extraction: 100%|██████████| 20/20 [00:17<00:00,  1.17it/s]


Fitting 5 folds for each of 105 candidates, totalling 525 fits
{'classifier': 'RandomForestClassifier', 'period_length': 2500, 'sampling_rate': 5000, 'accuracy': 0.9802890932982917, 'precision': 0.9804438736979363, 'recall': 0.9805451036390042, 'f1_score': 0.9804908359131773, 'best_params_': "{'criterion': 'entropy', 'max_depth': 40, 'n_estimators': 80}"}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\dante\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dante\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

C:\Users\dante\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dante\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

{'classifier': 'LogisticRegression', 'period_length': 2500, 'sampling_rate': 5000, 'accuracy': 0.9211563731931669, 'precision': 0.9222324576814516, 'recall': 0.9224471169695888, 'f1_score': 0.9222790837777428, 'best_params_': "{'C': 100}"}
Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'classifier': 'GaussianNB', 'period_length': 2500, 'sampling_rate': 5000, 'accuracy': 0.7174770039421814, 'precision': 0.7443198420614328, 'recall': 0.7266239460260326, 'f1_score': 0.657446544082718, 'best_params_': "{'var_smoothing': 0.001}"}
Fitting 5 folds for each of 45 candidates, totalling 225 fits
